## PDF Parsing

In [1]:
import sys
import os 
print(os.getcwd())

#sys.path.append('../src/')
#import src.ReadPDF as pdf_reader
import src.mapping as mapping
print(mapping.__file__)
# print(mapping in globals())
#sys.path.append('../src/classes')
import src.classes
from src.classes.PDFReader import PDFReader as pdf_reader
#import inspect
#print(inspect.getmodule(pdf_reader))
#print(sys.path)
from src.classes.Prompter import Prompter
#print(inspect.getmodule(Prompter))
import importlib
#print(dir(sys.modules))
#importlib.reload(pdf_reader)
# importlib.reload(ReadPDF)


/Users/natalieperrochon/CS180/new_clone/Partner-ESI-ML-Report
/Users/natalieperrochon/CS180/new_clone/Partner-ESI-ML-Report/src/mapping.py


In [2]:
print('src.classes.PDFReader' in sys.modules)
for key in sorted(sys.modules):
    #print(type(key))
    pass
#importlib.reload(pdf_reader)
importlib.reload(src.classes.PDFReader)
#importlib.reload(pdf_reader)

True


<module 'src.classes.PDFReader' from '/Users/natalieperrochon/CS180/new_clone/Partner-ESI-ML-Report/src/classes/PDFReader.py'>

In [3]:
# Extract by Section
#pdf_path = "20-301704.3.pdf" # ESA BL Data row 5736 - 6666
pdf_path = "4444_East.pdf" # ESA BL Data row 5736 - 6666

In [4]:
pdf_path2 = "20-301704.3.pdf" # ESA BL Data row 5736 - 6666


In [5]:
# Extract by Subsection
subsection_dict = pdf_reader().process_file(pdf_path)
#print(subsection_dict.keys()) #['1'][0].display_text())
#one_subsection = pdf_reader().extract_subsections(subsection_dict, 1, 1)
#print(one_subsection)

here
/Users/natalieperrochon/CS180/new_clone/Partner-ESI-ML-Report/src/classes/../../data/raw/4444_East.pdf
extracing by section .......................................................................................


In [6]:
subsection_dict = pdf_reader().process_file(pdf_path2)


here
/Users/natalieperrochon/CS180/new_clone/Partner-ESI-ML-Report/src/classes/../../data/raw/20-301704.3.pdf
extracing by section .......................................................................................


In [7]:
# Extract by Subsection
parser = pdf_reader()
subsection_dict = parser.process_file(pdf_path)
#subsection_dict

here
/Users/natalieperrochon/CS180/new_clone/Partner-ESI-ML-Report/src/classes/../../data/raw/4444_East.pdf
extracing by section .......................................................................................


## LLM Setup + Prompt

In [8]:
prompter = Prompter()

In [9]:
%%time
#response = prompter.invoke('Name of Assessor/Appraisal District Agency', one_subsection)
datafields = mapping.execute()
#datafields.items()
# for subsection in sorted(datafields.items()):
#     print(f"-----------Section_num: {subsection}--------")
#     print(subsection, datafields.get_section_fields(subsection))
    
    # response = prompter.invoke(datafields[subsection.items()[1], 
#print(response)

An error occurred: [Errno 2] No such file or directory: '/Users/natalieperrochon/CS180/new_clone/Partner-ESI-ML-Report/../data/raw/PCA_FIELDS.xlsx'


TypeError: 'NoneType' object is not subscriptable

In [43]:
#datafields

In [11]:
from langchain_community.llms import Ollama
llm = Ollama(model="phi", temperature=0, num_predict=40, top_k=5, top_p=.5, mirostat_tau=0, format="json")

In [12]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.output_parsers import JsonOutputParser

output_parser = StrOutputParser()
json_parser = JsonOutputParser()

In [13]:
%%time
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate.from_template("""Retrieve the datafield defined in input within the context given.
Anything legal can be found in the context.
If you cannot answer the question with the context, respond with N/A.

<context>
{context}
</context>

{input} is the key in the dictionary where the value is the prompt for the datafield.
Simply respond with only the answer that matches the datafield.
There must only be one JSON object.
Do not include the datafield in your response. Keep the key for the json exactly the same as the input.
""")

document_chain = create_stuff_documents_chain(llm, prompt, output_parser=json_parser)

CPU times: user 465 µs, sys: 7 µs, total: 472 µs
Wall time: 490 µs


## Responses

In [14]:
fields_dict = {'ets_2p_32a': 'Name of Assessor/Appraisal District Agency', 
               'ets_2p_33a': 'Property Legal Description', 
               'ets_2p_34a': 'Property Owner Name'}
#               'ets_2p_35a': "When did the owner acquire the property?",
 #              "ets_2p_36a": "What was the source of the property owner name and acquisition date?"}

In [15]:
#mappings = mapping.execute()

In [16]:
#mappings
# {subsection_num: {field: question}}

In [17]:
%%time
from langchain_core.documents import Document
answers_dict = {}
for items in datafields.items():
    try:
        item = {items[0]: items[1]}
    except Exception as e:
        print('ERROR')
        print(items)
        print(e)
        print()
    # print(type(item))
    #print(item)
    # print()
    for subsection in item:
        for field in item[subsection]:
            # print(field)
            # print(item[subsection][field])
            # for key in subsection_dict.keys():
            #     print('.' + key + '.')
            #     print(type(key))
                
            print(subsection_dict.keys())
            print(' 3' in subsection_dict.keys())
            print(item[subsection][field])
            subsection_context = pdf_reader.select_Subsection(subsection_dict, int(subsection[0]), subsection) #ERROR 
            output_dict = document_chain.invoke({
                "input": item[subsection][field],
                "context": [Document(page_content=subsection_context)]
            })
            key = field
            print(output_dict)
            print(output_dict.keys[0])
            #answers_dict[field] = output_dict[item[subsection][field]]
            #print(answers_dict[field])
print(answers_dict)

NameError: name 'datafields' is not defined

In [18]:
%%time
from langchain_core.documents import Document

print(type(document_chain.invoke({
    "input": "Property Legal Description",
    "context": [Document(page_content=one_subsection)]
})))

NameError: name 'one_subsection' is not defined

In [19]:
%%time
from langchain_core.documents import Document

document_chain.invoke({
    "input": "Property Owner Name",
    "context": [Document(page_content=one_subsection)]
})

NameError: name 'one_subsection' is not defined

In [ ]:
%%time
from langchain_core.documents import Document

document_chain.invoke({
    "input": "When did the owner acquire the property?",
    "context": [Document(page_content=one_subsection)]
})

In [ ]:
%%time
from langchain_core.documents import Document

document_chain.invoke({
    "input": "What was the source of the property owner name and acquisition date?",
    "context": [Document(page_content=one_subsection)]
})